# XGBoost Classification

**Classification whether a given ride was taken by a member or daypass-user.**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
%matplotlib inline

In [2]:
finale = pd.read_csv('finale1.csv', error_bad_lines=False, parse_dates=['date', 'starttime','stoptime' ], infer_datetime_format=True)
pd.set_option('display.max_columns', 50) 

In [3]:
finale['year'] = pd.DatetimeIndex(finale['date']).year
finale['month'] = pd.DatetimeIndex(finale['date']).month # add month as column
finale['hour'] = pd.DatetimeIndex(finale['starttime']).hour # add hour
finale['weekday'] = (finale['day'] < 5).astype(float) # add weekday or not

In [4]:
finale.head()

,trip_id,starttime,stoptime,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,birthyear,minutes,date,hour,day,from_lat,from_long,from_dockcount,to_lat,to_long,to_dockcount,max_temperature_f,mean_temperature_f,min_temperaturef,max_dew_point_f,meandew_point_f,min_dewpoint_f,max_humidity,mean_humidity,min_humidity,max_sea_level_pressure_in,mean_sea_level_pressure_in,min_sea_level_pressure_in,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_speed_mph,mean_wind_speed_mph,precipitation_in,events,year,month,weekday
0,431,2014-10-13 10:31:00,2014-10-13 10:48:00,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1960.0,16.432250,2014-10-13,10,0,47.60595,-122.335768,18,47.600757,-122.332946,18,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,0.0,Rain,2014,10,1.0
1,432,2014-10-13 10:32:00,2014-10-13 10:48:00,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1970.0,15.439583,2014-10-13,10,0,47.60595,-122.335768,18,47.600757,-122.332946,18,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,0.0,Rain,2014,10,1.0
2,433,2014-10-13 10:33:00,2014-10-13 10:48:00,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1988.0,14.730517,2014-10-13,10,0,47.60595,-122.335768,18,47.600757,-122.332946,18,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,0.0,Rain,2014,10,1.0
3,434,2014-10-13 10:34:00,2014-10-13 10:48:00,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1977.0,14.432283,2014-10-13,10,0,47.60595,-122.335768,18,47.600757,-122.332946,18,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,0.0,Rain,2014,10,1.0
4,435,2014-10-13 10:34:00,2014-10-13 10:49:00,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1971.0,15.398717,2014-10-13,10,0,47.60595,-122.335768,18,47.600757,-122.332946,18,71,62.0,54,55,51,46,87,68,46,30.03,29.79,29.65,10,10,4,13,4,0.0,Rain,2014,10,1.0


In [5]:
finale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236044 entries, 0 to 236043
Data columns (total 43 columns):
trip_id                       236044 non-null int64
starttime                     236044 non-null datetime64[ns]
stoptime                      236044 non-null datetime64[ns]
tripduration                  236044 non-null float64
from_station_name             236044 non-null object
to_station_name               236044 non-null object
from_station_id               236044 non-null object
to_station_id                 236044 non-null object
usertype                      236044 non-null object
gender                        146156 non-null object
birthyear                     146152 non-null float64
minutes                       236044 non-null float64
date                          236044 non-null datetime64[ns]
hour                          236044 non-null int64
day                           236044 non-null int64
from_lat                      236044 non-null float64
from_long       

In [6]:
# usertype replace
finale.replace(to_replace=['Member', 'Short-Term Pass Holder'], value=[0, 1], inplace=True)

In [7]:
finale.drop(['starttime', 'stoptime', 'from_station_name', 'to_station_name', 'from_station_id', 'to_station_id',
             'gender', 'birthyear', 'date', 'events' ], axis = 1, inplace = True)

In [8]:
finale.shape

(236044, 33)

In [9]:
finale['usertype'] = pd.to_numeric(finale['usertype'])

In [10]:
finale.usertype.nunique

<bound method IndexOpsMixin.nunique of 0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         1
9         0
10        1
11        1
12        0
13        1
14        1
15        1
16        1
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        1
26        0
27        0
28        0
29        1
         ..
236014    1
236015    1
236016    1
236017    1
236018    1
236019    1
236020    1
236021    1
236022    1
236023    1
236024    1
236025    1
236026    1
236027    1
236028    1
236029    1
236030    0
236031    0
236032    1
236033    1
236034    0
236035    1
236036    1
236037    1
236038    1
236039    1
236040    1
236041    1
236042    1
236043    1
Name: usertype, Length: 236044, dtype: int64>

In [ ]:
# export csv for neural network

In [11]:
# scale impute

In [12]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn import preprocessing

import xgboost as xgb
from xgboost import XGBClassifier

In [13]:
y1 = finale.usertype
x1 = finale.drop(columns=['usertype'])

x_train, x_test, y_train, y_test = train_test_split(x1,y1, test_size=0.25, random_state=10)

model = XGBClassifier()
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)

In [14]:
from sklearn.model_selection import cross_val_score
recall = cross_val_score(model, x_train, y_train, cv=5, scoring='recall')
print('Recall', np.mean(recall), recall)
precision = cross_val_score(model, x_train, y_train, cv=5, scoring='precision')
print('Precision', np.mean(precision), precision)
f1 = cross_val_score(model, x_train, y_train, cv=5, scoring='f1')
print('F1', np.mean(f1), f1)

Recall 0.7453912785962834 [0.74420671 0.74627971 0.74220774 0.74494706 0.74931517]
Precision 0.8303342553678558 [0.83474506 0.83250743 0.82830703 0.82522759 0.83088416]
F1 0.7855691353233116 [0.78688011 0.78703884 0.78289731 0.78303502 0.78799439]


In [12]:
y_pred_xgb=model.predict(x_test)
print(classification_report(y_test, y_pred_xgb))

                        precision    recall  f1-score   support

                Member       0.85      0.91      0.88     36658
Short-Term Pass Holder       0.83      0.75      0.79     22353

              accuracy                           0.85     59011
             macro avg       0.84      0.83      0.83     59011
          weighted avg       0.84      0.85      0.84     59011



# Conclusion